# Genetic Algorithm Assignment
30% of the overall grade for this module

Marks indciated in sections below are based on percentage of marks allocated for this module

In this assignment you must choose a problem, and attempt to use the Genetic Alogrithm that we developed in class to solve this problem.



## The Problem         **(~30%)**

*   Description of the problem

## Improving the Game Snake via the Genetic Algorithm
This project aims to record the input of a game of Snake and assess the outputs of said game using a Genetic Algorithm. The goal here is to optimise the game itself to improve the score. With the scope of the board size and the sheer amount of possible moves to be had, I don't think it is realistically possible to win the game in any circumstance.

Previous iterations of this project were able to improve the score to 1, so the goal for this project should naturally be to get a score of greater than or above 2.


*   Discussion of the suitablity of Genetic Algorithms

## Suitability of the Genetic Algorithm
A Genetic Algorithm is uniquely suited to this problem, as the game as it exists here is a series of recordable inputs that can be quantified into a possible score function.
As a note here, the Output for the Snake program and Input into the GA will look like the following:
```python
{['a','w','d','s'], score=2}
```

1. ### Inputs and Outputs
The Snake game made here is a good candidate for the Genetic Algorithm as it has easily quantifiable metrics that could be used for improvement.
* **Moves:** The moves here are going to be the main input in Snake, and defines the other metrics such as score, because score cannot be incremented if the player doesn't move.
* **Score:** The score metric of the game corresponds to the amount of food eaten, and the length of the snake. Here, it is recorded as "Score".

2. ### Crossover:
The Crossover will be responsible for making new chromosomes from the old ones. This is generally coded as taking the before and after of some point in a chromosome. This is easily done with the series of moves and scores given when the program is run according to a random seed.

3. ### Mutation:
The Mutation here will be responsible for introducing new, random changes into a chromosome. This will introduce diversity and new combinations into the population, and prevent it from getting stale, allowing for exploring new solutions.

4. ### Iterative Improvement
The GA will improve the results over subsequent generations, making the program better with each output.

5. ### Scalability
This program is easily scalable, and should be easily increased and assessed by running the game more times and increasing the population size for the GA.

*   Complexity of the problem  (Overall marks allocated based on ..)
---

# The problem **(~20%)**

<hr>

### Problem Parameters:
The GA in this case will have the following parameters associated with it:
* Population Size: The population size is the number of times the problem is run and how many chromosomes are generated.
* Generations: This refers to the number of iterations required to evolve the best solution.
* Mutation Rate: This is the number of times a solution is changed over the course of a single generation.

In [1]:
import random
import numpy as np
from copy import deepcopy

In [2]:
class Snake: # Code for the Snake Game in this cell.
    def __init__(self, chromosome=None):
        self.board = [
          ['#', '-', '-', '-', '-', '-', '-', '#'],
          ['|', ' ', ' ', ' ', ' ', ' ', ' ', '|'],
          ['|', ' ', ' ', ' ', ' ', ' ', ' ', '|'],
          ['|', ' ', ' ', ' ', ' ', ' ', ' ', '|'],
          ['|', ' ', ' ', ' ', ' ', ' ', ' ', '|'],
          ['#', '-', '-', '-', '-', '-', '-', '#']
        ]
        self.snake = [(3, 3)] # Hard coded starting position
        self.direction = 'w' # Always starts facing up
        self.moves = []
        self.height = len(self.board)
        self.width = len(self.board[0])
        self.score = 0
        self.chromosome = chromosome
        self.place_snake()
        self.generate_food()

    def place_snake(self):
        head_x, head_y = self.snake[0]
        self.board[head_x][head_y] = '0'

    def generate_food(self):
        for i in range(self.height):
            for j in range(self.width):
                if self.board[i][j] == '@':
                    self.board[i][j] = ' '
        valid_positions = []
        for i in range(1, self.height - 1):
            for j in range(1, self.width - 1):
                if (i, j) not in self.snake and self.board[i][j] == ' ':
                    valid_positions.append((i, j))

        if valid_positions:
            food_x, food_y = random.choice(valid_positions)
            self.board[food_x][food_y] = '@'
            return (food_x, food_y)
        else:
            return None

    def display_board(self):
        for row in self.board:
            print(' '.join(str(cell) for cell in row))

    def accept_input(self, chromosome, move_index):
        if move_index < len(chromosome):
            move_value = chromosome[move_index]
            if 0 <= move_value < 1:
                move = 'w'
            elif 1 <= move_value < 2:
                move = 's'
            elif 2 <= move_value < 3:
                move = 'a'
            elif 3 <= move_value <= 4:
                move = 'd'
            else:
                move = self.direction
            self.moves.append(move)
            return move # If invalid move, keep moving in same direction.

    def move_snake(self):
        head_x, head_y = self.snake[0]
        if self.direction == "w":
            new_head = (head_x - 1, head_y) # Up
        elif self.direction == "s":
            new_head = (head_x + 1, head_y) # Down
        elif self.direction == "a":
            new_head = (head_x, head_y - 1) # Left
        elif self.direction == "d":
            new_head = (head_x, head_y + 1) # Right

        new_x, new_y = new_head # Validate head
        if (self.board[new_x][new_y] == '#' or
            self.board[new_x][new_y] == '-' or
            self.board[new_x][new_y] == '|' or
            new_head in self.snake):
            return False

        food_eaten = self.board[new_x][new_y] == '@'

        self.snake.insert(0, new_head)
        self.board[new_x][new_y] = '0'

        if len(self.snake) > 1:
            prev_head_x, prev_head_y = self.snake[1]
            self.board[prev_head_x][prev_head_y] = 'o'

        if not food_eaten:
            tail_x, tail_y = self.snake.pop()
            self.board[tail_x][tail_y] = ' '
        else:
            self.score += 1
            self.generate_food()
        return True

    def play_game(self, chromosome):
        game_over = False
        move_index = 0
        while not game_over:
            self.direction = self.accept_input(chromosome, move_index)
            game_over = not self.move_snake()
            move_index += 1

        print(f"The result is: {self.moves}, Score: {self.score}")
        return self.score, self.moves

## Genetic Algorithm Class
This defines a Genetic algorithm class constructor along with it's parameters.
- **Elitism Count:** Best selections from each generation.
- **Generation Stats:** Maximum, Minimum and Average fitness will be tracked and sanity checked.

In [3]:
class GeneticAlgorithm:
    def __init__(self, population_size, chromosome_length, mutation_rate, crossover_rate, elitism_count):
        self.population_size = population_size
        self.chromosome_length = chromosome_length
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.elitism_count = elitism_count
        self.generation_stats = []
        self.best_chromosome = None
        self.best_fitness = -1 # Initialise to a negative value

# The Individual **(~30%)**

## Individual
An individual represents a single instance of the game and it's output.
An individual contains:
- The moves made by the game.
- The score taken at the end.

<hr>

* **Chromosome:** Also known as the individual, this will correspond to the program outputs for each iteration. It is a series of movements made by the program, and the recorded score.
* **Population:** The number of chromosomes to be assessed each generation.

<hr>


In [4]:
def create_individual(self):
    return [random.uniform(0, 4) for _ in range(self.chromosome_length)]

def create_population(self):
    return [self.create_individual() for _ in range(self.population_size)]
GeneticAlgorithm.create_individual = create_individual # Adds methods to classes after declaration, allowing for access later, in different cells
GeneticAlgorithm.create_population = create_population

## Cost Function
The cost function needs to be able to assess how well a chromosome performed.

- **Score:** The score is the most important factor, as the only way to win the game is to maximise the score.
- **Length:** The mount of time the chromosome stays alive, needs to be handled differently than the score, as an individual could stay allive by going in circles, so this factor should have less weighting than the Score.

In [5]:
# Cost Function
def cost_function(self, chromosome):
    snake_game = Snake(chromosome)
    score, moves = snake_game.play_game(chromosome)

    base_fitness = score * 100  # Heavy weight on eating food, less on survival time.
    survival_bonus = len(moves) * 1

    if len(moves) > 0: # Calculate chromosome efficiency.
        efficiency_bonus = (score / len(moves)) * 50
    else:
        efficiency_bonus = 0
    total_fitness = base_fitness + survival_bonus + efficiency_bonus
    return max(0, total_fitness)

    print(f"Error evaluating fitness: {e}")
    return 0
GeneticAlgorithm.cost_function = cost_function

## Discussion and justification on the approaches taken for the above

## Selection Function
Choosing the parents is a process resulting from the fitness assessment of each individual. This program will use elitism to carry over the best chromosomes from each set.

### Tournament Selection
The Tournament selection method returns the most efficient chromosomes. It's very verbose, but supports changing tournament valuies, and shoule demonstrate the

### Crossover
The crossover function defines where a new chromosome should be created from the old ones. The array should exist as a point that needs to be split somewhere, retaining part one and part two of each parent.

### Mutation
The Mutation function introduces random samples into a chromosome. It inserts a uniform selection of random moves into an existing chromosome to introduce  novelty to the genes.

In [6]:
def tournament_selection(self, population, fitnesses, tournament_size=3):
    tournament_indices = random.sample(range(len(population)), tournament_size)
    tournament_fitnesses = [fitnesses[i] for i in tournament_indices]
    winner_index = tournament_indices[tournament_fitnesses.index(max(tournament_fitnesses))]
    return population[winner_index]
GeneticAlgorithm.tournament_selection = tournament_selection

def crossover(self, parent1, parent2):
    if random.random() > self.crossover_rate:
        return parent1[:], parent2[:]

    crossover_point = random.randint(1, len(parent1) - 1)

    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]

    return child1, child2
GeneticAlgorithm.crossover = crossover

def mutate(self, chromosome):
    mutated = chromosome[:]
    for i in range(len(mutated)):
        if random.random() < self.mutation_rate:
            mutated[i] = random.uniform(0, 4)

    return mutated
GeneticAlgorithm.mutate = mutate

# Evolution
The `evolve_generation()` method here takes the best individuals from the program and carries them into the next generation. This ensures the best chromosomes are preserved.

Then it performs crossover and mutation on the parents to create offspring to introduce genetic diversity in the population.

Then it returns the new population.

In [7]:
def evolve_generation(self, population, fitnesses):
    new_population = []

    elite_indices = sorted(range(len(fitnesses)), key=lambda i: fitnesses[i], reverse=True)[:self.elitism_count]
    for i in elite_indices:
        new_population.append(population[i][:])

    while len(new_population) < self.population_size: # Crossover and Mutation to generate children
        parent1 = self.tournament_selection(population, fitnesses)
        parent2 = self.tournament_selection(population, fitnesses)

        child1, child2 = self.crossover(parent1, parent2)

        child1 = self.mutate(child1)
        child2 = self.mutate(child2)

        new_population.extend([child1, child2])

    return new_population[:self.population_size]
GeneticAlgorithm.evolve_generation = evolve_generation

## Running the algorithm  **(~10%)**

#### Parameter Choices
1. Population Size - The size of the given population per generation. 100, in this case.
2. Chromosome Length - The amount of moves that were made over the programs duration. 150 as the maximum value. Because of how Snake works, it is very unlikely to reach 150 before failing some condition.
3. Mutation Rate - The amount at which each chromosome has random changes added to it. This ensures novelty every instance. 0.15 in this rate should introduce random changes into 15% of chromosomes.
4. Crossover Rate - The crossover rate defines how often the parents are used to create children. At the rate of 0.8 means that the parents will be used to create solutions 80% of the time. Parents are directly preserved 20% of the time.
5. Elitism Count - This parameter defines how many iterations are directly preserved for the next generation. In this case, I kept it around 10% of the population size.

#### Run Genetic:
The run genetic method here performs the same purpose as the original, but has been edited to print generational metrics. To that end, the generations are also defined here instead of . It displays average fitness, maximum fitness and minimum fitness for each generation.


In [8]:
def run_genetic(self, generations=100):
    population = self.create_population()

    for generation in range(generations):
        fitnesses = [] # Array of fitness called FITNESSES. Used for evolution
        for individual in population:
            fitness = self.cost_function(individual)
            fitnesses.append(fitness)

        max_fitness = max(fitnesses)
        avg_fitness = np.mean(fitnesses)
        min_fitness = min(fitnesses)

        best_idx = fitnesses.index(max_fitness)
        if max_fitness > self.best_fitness:
            self.best_fitness = max_fitness
            self.best_chromosome = population[best_idx][:]

        self.generation_stats.append({
            'generation': generation,
            'max_fitness': max_fitness,
            'avg_fitness': avg_fitness,
            'min_fitness': min_fitness
        })

        print(f"Generation {generation}: Max={max_fitness}, " f"Avg={avg_fitness}, Min={min_fitness}")

        population = self.evolve_generation(population, fitnesses)

    return self.best_chromosome, self.best_fitness
GeneticAlgorithm.run_genetic = run_genetic

In [9]:
def main():
    print("Starting Genetic Algorithm Evolution for Snake Game")

    ga = GeneticAlgorithm( # Modify parameters here.
        population_size=100,
        chromosome_length=150, # Max length
        mutation_rate=0.15, # 15% mutation rate
        crossover_rate=0.8, # 80% crossover rate
        elitism_count=10 # Keep 10 samples from generation
    )

    best_chromosome, best_fitness = ga.run_genetic(generations=10)
    print(f"Best fitness achieved: {best_fitness:.1f}")
    return ga

ga = main()

Starting Genetic Algorithm Evolution for Snake Game
The result is: ['w', 'a', 'd', 'w', 'd', 'a', 'w'], Score: 0
The result is: ['s', 'w', 'w', 'd', 'w', 'a', 's', 'a', 'd', 's', 'a', 'a', 'd', 'a', 'a'], Score: 0
The result is: ['d', 'a', 'd', 's', 's'], Score: 0
The result is: ['d', 's', 'd', 's'], Score: 0
The result is: ['w', 'd', 's', 'a', 'w', 'd', 'a', 'd', 's', 's', 'w', 'd', 's', 'a', 'w', 's'], Score: 1
The result is: ['w', 'w', 'd', 'a', 's', 'w', 'w'], Score: 0
The result is: ['w', 'a', 'a', 'a'], Score: 0
The result is: ['w', 'a', 'w', 'a', 's', 'd', 'd', 's', 's', 'a', 's'], Score: 0
The result is: ['s', 'd', 'w', 'd', 's', 'w'], Score: 1
The result is: ['d', 'a', 's', 'a', 'a', 's'], Score: 0
The result is: ['w', 'a', 'd', 'w', 'w'], Score: 0
The result is: ['w', 'w', 's', 's', 's', 'w', 'd', 'a', 'd', 'a', 'a', 'w', 'w', 'a', 'a'], Score: 0
The result is: ['w', 'w', 'd', 'w'], Score: 0
The result is: ['w', 'd', 'd', 'a'], Score: 1
The result is: ['a', 'd', 'a', 'd', 'w'

## Results and conclusions    **(~10%)**

---

The work done here has demonstrated that the Genetic Algorithm can be used to enhance the Snake Game going forward. The resulting algorithm has produced better results than previous iterations of the same project, working as intended. Previous iterations were often not able to get a higher score than 1. This iteration occasionally gets scores as high as 3, and a score of 2 is much more common.

Future iterations of this project might focus moreso on fine tuning the parameters for the algorithm to enhance it on a smaller scale. The algorithm itself could be modified with a better seed generation, as a real game would not work with set seeds.

---

